In [29]:
import pandas as pd
import numpy as np
import configparser
import nltk
import sys
from nltk.stem.porter import PorterStemmer
config = configparser.ConfigParser()
config.read("../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']
sys.path.append(local_lib)

OHCO = ['para_num', 'sent_num', 'token_num']

In [30]:
LIB = pd.read_csv('LIB.csv', sep='|').set_index('speech_id')
CORPUS = pd.read_csv('CORPUS.csv', sep='|').set_index(['speech_id'] + OHCO)
DTM = pd.read_csv('DTM.csv').set_index(['speech_id', 'para_num'])

In [31]:
CORPUS = CORPUS[CORPUS.term_str!='']
CORPUS['pos_group'] = CORPUS.pos.str[:2]

VOCAB = CORPUS.term_str.value_counts().to_frame('n').sort_index()
VOCAB.index.name = 'term_str'
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB.n / VOCAB.n.sum()
VOCAB['i'] = -np.log2(VOCAB.p)
VOCAB['max_pos'] = CORPUS[['term_str', 'pos']].value_counts().unstack(fill_value=0).idxmax(1)
VOCAB['max_pos_group'] = CORPUS[['term_str', 'pos_group']].value_counts().unstack(fill_value=0).idxmax(1)

In [32]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

VOCAB['stop'] = VOCAB.index.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype(int)

stemmer1 = PorterStemmer()
VOCAB['porter_stem'] = VOCAB.apply(lambda x: stemmer1.stem(x.name), 1)

In [33]:
DF = DTM.astype('bool').sum()
N = DTM.shape[0]
IDF = np.log2(N / DF)
VOCAB['dfidf'] = DF * IDF
VOCAB

,n,n_chars,p,i,max_pos,max_pos_group,stop,porter_stem,dfidf
term_str,,,,,,,,,
04,1,2,0.000002,19.052799,CD,CD,0,04,13.198292
07,1,2,0.000002,19.052799,CD,CD,0,07,13.198292
08,1,2,0.000002,19.052799,CD,CD,0,08,13.198292
09,1,2,0.000002,19.052799,CD,CD,0,09,13.198292
1,153,1,0.000281,11.795411,CD,CD,0,1,849.661195
...,...,...,...,...,...,...,...,...,...
zion,1,4,0.000002,19.052799,NNP,NN,0,zion,13.198292
zone,6,4,0.000011,16.467836,NN,NN,0,zone,54.381817
zones,18,5,0.000033,14.882874,NNS,NN,0,zone,147.172665


In [34]:
VOCAB.to_csv('VOCAB.csv', sep='|')

In [37]:
VOCAB[VOCAB['stop']==0].sort_values(by='dfidf', ascending=False).head(20)

,n,n_chars,p,i,max_pos,max_pos_group,stop,porter_stem,dfidf
term_str,,,,,,,,,
people,1924,6,0.003538,8.142906,NNS,NN,0,peopl,4049.914776
must,1934,4,0.003556,8.135427,MD,MD,0,must,3942.122274
us,1608,2,0.002957,8.401747,PRP,PR,0,us,3684.610313
world,1602,5,0.002946,8.407140,NN,NN,0,world,3667.352042
new,1578,3,0.002902,8.428917,JJ,JJ,0,new,3624.966156
congress,1395,8,0.002565,8.606749,NNP,NN,0,congress,3578.575270
america,1412,7,0.002596,8.589274,NNP,NN,0,america,3534.140499
american,1372,8,0.002523,8.630734,JJ,JJ,0,american,3469.559728
year,1367,4,0.002514,8.636001,NN,NN,0,year,3369.514586
